In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df=pd.read_csv('CKD.csv')
df=pd.get_dummies(df,dtype=int,drop_first=True)
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [3]:
df.isnull().values.any()

np.False_

In [4]:
indep=df[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=df['classification_yes']

In [5]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(indep)

DECISION_TREE

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid={'criterion':['log_loss', 'entropy', 'gini'],
              'max_features': [None,'sqrt','log2'],
              'max_depth': [None, 10, 20],  # Use max_depth instead of max_features
    'min_samples_split': [2, 10]}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(indep,dep)



Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['log_loss', 'entropy', 'gini'],
                         'max_depth': [None, 10, 20],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_split': [2, 10]},
             scoring='f1_weighted', verbose=3)

In [9]:
re=grid.cv_results_
y_pred=grid.predict(indep)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(dep,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(dep,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_auc_score=(dep,grid.predict_proba(indep)[:,1])

[[150   0]
 [  4 245]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       150
           1       1.00      0.98      0.99       249

    accuracy                           0.99       399
   macro avg       0.99      0.99      0.99       399
weighted avg       0.99      0.99      0.99       399

The f1_macro value for the best parameters {'criterion': 'log_loss', 'max_depth': 20, 'max_features': 'log2', 'min_samples_split': 10}: 0.9900003044047364


In [10]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.043192,0.045665,0.022809,0.033128,log_loss,None,None,2,"{'criterion': 'log_loss', 'max_depth': None, '...",0.452683,0.950296,0.936147,0.987539,1.000000,0.865333,0.207646,37
1,0.025074,0.030024,0.019146,0.026881,log_loss,None,None,10,"{'criterion': 'log_loss', 'max_depth': None, '...",0.452683,0.938022,0.936147,0.987539,0.987380,0.860354,0.205078,45
2,0.005652,0.002997,0.018650,0.030939,log_loss,None,sqrt,2,"{'criterion': 'log_loss', 'max_depth': None, '...",0.876918,0.987539,0.975000,0.987539,0.987380,0.962875,0.043250,11
3,0.003431,0.003083,0.006055,0.002914,log_loss,None,sqrt,10,"{'criterion': 'log_loss', 'max_depth': None, '...",0.507812,0.949628,0.949628,0.987539,0.987298,0.876381,0.185058,27
4,0.007043,0.002022,0.034982,0.035517,log_loss,None,log2,2,"{'criterion': 'log_loss', 'max_depth': None, '...",0.532378,0.962368,0.923764,0.975148,0.503120,0.779356,0.214469,54
5,0.001818,0.002224,0.021879,0.029667,log_loss,None,log2,10,"{'criterion': 'log_loss', 'max_depth': None, '...",0.840062,0.962813,0.974814,0.962813,0.555323,0.859165,0.159715,50
6,0.007055,0.003001,0.021175,0.032449,log_loss,10,None,2,"{'criterion': 'log_loss', 'max_depth': 10, 'ma...",0.452683,0.950296,0.936147,0.987539,1.000000,0.865333,0.207646,37
7,0.029510,0.027218,0.004949,0.002676,log_loss,10,None,10,"{'criterion': 'log_loss', 'max_depth': 10, 'ma...",0.452683,0.938022,0.936147,0.987539,1.000000,0.862878,0.206697,41
8,0.035201,0.040553,0.005034,0.003807,log_loss,10,sqrt,2,"{'criterion': 'log_loss', 'max_depth': 10, 'ma...",0.559100,0.987539,0.923764,0.975148,0.514868,0.792084,0.209850,53
9,0.020123,0.030307,0.002717,0.002373,log_loss,10,sqrt,10,"{'criterion': 'log_loss', 'max_depth': 10, 'ma...",0.912192,1.000000,0.975000,0.962813,0.791083,0.928218,0.074298,23
